<img style="float: left;;" src='Figures/iteso.jpg' width="100" height="200"/></a>

# <center> <font color= #000047> Datos Atípicos </font> </center>

## ¿Qué son los valores atípicos (outliers)?

Un valor atípico es una observación que es diferente a las demás. Es raro, o distinto, o no encaja de alguna manera. También se les llama anomalías.

Los valores atípicos pueden suceder por varias causas, tales como:

- Error de medición o entrada.
- Corrupción de datos.
- Observación verdaderamente atípica.

Incluir valores atípicos en modelos basados en datos puede ser riesgoso. La existencia de un solo valor extremo engañoso tiene el potencial de cambiar la conclusión que implica el modelo. Por lo tanto, es importante gestionar ese tipo de riesgo.

> No hay una forma precisa de definir e identificar valores atípicos en general debido a las particularidades de cada conjunto de datos. En su lugar, se puede interpretar las observaciones y decidir si un valor es atípico o no.

> Podemos usar métodos estadísticos para identificar observaciones que parecen ser raras o poco probables dado los datos disponibles. Esto no significa que los valores identificados sean atípicos y deban eliminarse.

> Un buen consejo es considerar graficar los valores atípicos identificados, quizás en el contexto de los valores no atípicos para ver si hay relaciones o patrones sistemáticos. Si los hay, tal vez no sean atípicos y puedan explicarse, o tal vez los propios valores atípicos puedan identificarse de manera más sistemática.

#### Tipos de Valores atípicos

Un valor atípico puede ser de dos tipos:

1. Univariado
2. Multivariado

Los valores atípicos univariados pueden encontrarse al observar la distribución de una sola variable. Los valores atípicos multivariados son valores atípicos en un espacio n-dimensional. Para encontrarlos, debes observar distribuciones en varias dimensiones.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
N = 20
x = np.linspace(0, 10, N)
y = 10 + 2 * x + np.random.normal(loc=0, scale=2, size=(N,))
y[0] = 30
y[-1] = 10

df_xy = pd.DataFrame({'x': x, 'y': y})
#distribuciones de x, y
plt.figure(figsize=(6,4))
df_xy.boxplot(figsize=(6,5));
plt.show()

In [ ]:
# Relación entre x, y
plt.figure(figsize=(6,4))
plt.plot(x,y,'xr', label='datos')
plt.grid()
plt.show()

Existen diversas técnicas para identificar valores atípicos en un conjunto de datos, incluyendo métodos de **inspección visual, métodos estadísticos e incluso modelos de machine learning como el Local Outlier Factor (LOF), el Isolation Forest (IForest)**.


### Outliers Univariados

Las metodologías más comunes para detectarlos son:

1. Método del Rango Intercuantílico (IQR)
2. Método de la desviación estándar
3. Método de la puntuación Z Z-score modificada

#### Método del Rango Intercuantílico (IQR)

El concepto de rango intercuartílico (IQR) se utiliza para construir los diagramas de caja. El IQR es un concepto en estadística que se utiliza para medir la dispersión y variabilidad de los datos dividiendo el conjunto en cuartiles.

Cualquier conjunto de datos u observaciones se divide en cuatro intervalos definidos según los valores de los datos y cómo se comparan con el conjunto completo. Un cuartil es lo que divide los datos en tres puntos y cuatro intervalos.

$IQR$ es la diferencia entre el tercer cuartil y el primer cuartil ($IQR = Q_3 - Q_1$). Los valores atípicos en este caso se definen como las observaciones que están por debajo de ($Q1 − 1.5*IQR$) o el bigote inferior del diagrama de caja, o por encima de ($Q3 + 1.5* IQR$) o el bigote superior. Puede representarse visualmente mediante el diagrama de caja.

In [ ]:
from sklearn.datasets import load_iris

LI=load_iris()
df=pd.DataFrame(LI.data,columns=LI.feature_names)
df.head()

In [ ]:
df.hist()
plt.show()

In [ ]:
df['sepal width (cm)'].values

In [ ]:
q3,q1=np.quantile(df,(0.75,0.25),axis=0)
iqr=q3-q1
iqr

In [ ]:
Li=q1-1.5*iqr
Ls=q3+1.5*iqr
Li

In [ ]:
Ls

In [ ]:
(df.iloc[:,1] < Li[1]) | (df.iloc[:,1] > Ls[1] ) #Outliers para sepal width (cm)

In [ ]:
spel_width_outliers = df[(df.iloc[:,1] < Li[1]) | (df.iloc[:,1] > Ls[1] )]['sepal width (cm)'] #Outliers para sepal length (cm)
spel_width_outliers

In [ ]:
spel_width_woutliers = df[~((df.iloc[:,1] < Li[1]) | (df.iloc[:,1] > Ls[1] ))]['sepal width (cm)']

In [ ]:
spel_width_woutliers.hist()
plt.show()

In [ ]:
def MetodoIQR (df,n,features):
    outlier_list = []
    
    for column in features:
                
        # 1st quartile (25%)
        Q1 = np.percentile(df[column], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[column],75)
        
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # limite iqr
        outlier_limit = 1.5 * IQR
        
        # determinar la lista de outliers
        outlier_list_column = df[(df[column] < Q1 - outlier_limit) | (df[column] > Q3 + outlier_limit )].index
        
        # agregando a la lista de outliers
        outlier_list.extend(outlier_list_column)
        
    # seleccionar las observaciones que contienen más de cierto numero de outliers
    outlier_list = Counter(outlier_list)
    print(outlier_list)
    multiple_outliers = list( k for k, v in outlier_list.items() if v >= n )
    
    return multiple_outliers

In [ ]:
outliers_index = MetodoIQR(df, 1, df.columns)
outliers_index

### Método de la desviación estándar

Si sabemos que la distribución de los valores en la muestra es gaussiana o similar a la gaussiana, podemos usar la desviación estándar de la muestra como un límite para identificar valores atípicos.

La desviación estándar muestra cuánto se dispersan los puntos de datos individuales respecto a la media. Si la distribución de los datos es normal entonces:
* El 68% de los valores de los datos se encuentran dentro de una desviación estándar de la media
* El 95% están dentro de dos desviaciones estándar
* El 99.7% se encuentran dentro de tres desviaciones estándar.

Dependiendo de la especificación establecida, ya sea a 2 o 3 veces la desviación estándar, podemos detectar y eliminar valores atípicos del conjunto de datos.

Este método puede fallar en la detección de valores atípicos porque los valores atípicos aumentan la desviación estándar. Cuanto más extremo sea el valor atípico, más se ve afectada la desviación estándar.

<img src="Figures/Standard_deviation_diagram.svg" width="800" height="800">


In [ ]:
# El conjunto de datos contiene transacciones realizadas con tarjetas de crédito. Contiene únicamente variables de entrada numéricas que son el resultado de una transformación PCA.
df_credit = pd.read_csv('Data/creditcard.csv') 
df_credit.head()

In [ ]:
df_credit.columns[:-1]

In [ ]:
feature_list = df_credit.columns[:-1].to_list()

In [ ]:
#Ploteamos los diagramas de caja de algunas características
fig, axes = plt.subplots(nrows=2, ncols=3,figsize=(12,8))
fig.suptitle('Diagrama de caja Características vs Class\n', size = 18)

sns.boxplot(ax=axes[0, 0], data=df, x='Class', y='V17')
axes[0,0].set_title("V17");

sns.boxplot(ax=axes[0, 1], data=df, x='Class', y='V10')
axes[0,1].set_title("V10");

sns.boxplot(ax=axes[0, 2], data=df, x='Class', y='V12')
axes[0,2].set_title("V12");

sns.boxplot(ax=axes[1, 0], data=df, x='Class', y='V16')
axes[1,0].set_title("V16");

sns.boxplot(ax=axes[1, 1], data=df, x='Class', y='V14')
axes[1,1].set_title("V14");

sns.boxplot(ax=axes[1, 2], data=df, x='Class', y='V3')
axes[1,2].set_title("V3");

plt.tight_layout()
plt.show()

In [ ]:
data_mean, data_std = df['V10'].mean(), df['V10'].std()
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off

print('The lower bound value is:', data_mean - cut_off)
print('The upper bound value is:', data_mean + cut_off)

plt.figure(figsize = (10,5))
sns.histplot(x = 'V11', data=df, bins=70)
plt.axvspan(xmin = lower,xmax= df.V10.min(),alpha=0.2, color='red')
plt.axvspan(xmin = upper,xmax= df.V10.max(),alpha=0.2, color='red')
plt.show()


In [ ]:
def Metodo_StDev(df,n,features):
    outlier_indices = []
    
    for column in features:
        # calcula la media y la desviación estándar de todo el dataframe
        data_mean = df[column].mean()
        data_std = df[column].std()
        
        # calcula el corte en la desviación estandar
        cut_off = data_std * 3
        
        # Determina la lista de indices de los outliers en cada columns
        outlier_list_column = df[(df[column] < data_mean - cut_off) | (df[column] > data_mean + cut_off)].index
        
        #Agrega los indices de los outliers obtenidos 
        outlier_indices.extend(outlier_list_column)
        
    # Selecciona las observaciones que contienen más de x outliers
    outlier_ind = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_ind.items() if v > n )
    
    return multiple_outliers   

In [ ]:
outliers_StDev = Metodo_StDev(df,1,df.columns) #Indices que representan outliers

In [ ]:
df_credit_wout = df.drop(outliers_StDev, axis = 0).reset_index(drop=True)
df_credit_wout.head()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3,figsize=(13,8))
fig.suptitle('Algunas distribuciones después de aplicar el método de desviación estandar\n', size = 18)

axes[0,0].hist(df_credit_wout['V17'], bins=60, linewidth=0.5, edgecolor="black")
axes[0,0].set_title("V17");

axes[0,1].hist(df_credit_wout['V10'], bins=60, linewidth=0.5, edgecolor="black")
axes[0,1].set_title("V10");

axes[0,2].hist(df_credit_wout['V12'], bins=60, linewidth=0.5, edgecolor="black")
axes[0,2].set_title("V12");

axes[1,0].hist(df_credit_wout['V16'], bins=60, linewidth=0.5, edgecolor="black")
axes[1,0].set_title("V16");

axes[1,1].hist(df_credit_wout['V14'], bins=60, linewidth=0.5, edgecolor="black")
axes[1,1].set_title("V14");

axes[1,2].hist(df_credit_wout['V3'], bins=60, linewidth=0.5, edgecolor="black")
axes[1,2].set_title("V3");

axes[2,0].hist(df_credit_wout['V7'], bins=60, linewidth=0.5, edgecolor="black")
axes[2,0].set_title("V7");

axes[2,1].hist(df_credit_wout['V11'], bins=60, linewidth=0.5, edgecolor="black")
axes[2,1].set_title("V11");

axes[2,2].hist(df_credit_wout['V4'], bins=60, linewidth=0.5, edgecolor="black")
axes[2,2].set_title("V4");

plt.tight_layout()
plt.show()

### Método de puntuación Z (Z-score)

Al calcular el **Z-Score** (puntuación Z) se utiliza para convertir los datos en otro conjunto con media = 0, describe la posición de un valor bruto en términos de su distancia respecto a la media, medida en unidades de desviación estándar.

Esta técnica asume una **distribución gaussiana** de los datos. Los valores atípicos serán los datos que están en las colas de la distribución. En la mayoría de los casos se utiliza un umbral de 3 o -3, es decir, si el valor de la puntuación Z es mayor o menor que 3 o -3 respectivamente, ese punto de datos será identificado como valor atípico.

In [ ]:
df_taxis = sns.load_dataset('taxis')
df_taxis.head()

In [ ]:
# Usando un método de visualización (histograma)
plt.figure(figsize=(5,4))
sns.histplot(data=df_taxis, x = 'total', bins=60, linewidth=0.5, edgecolor="black")
plt.show()

In [ ]:
z_score =  (df_taxis['total'] - df_taxis['total'].mean())/df_taxis['total'].std()
z_score

In [ ]:
plt.figure(figsize=(5,4))
sns.histplot(data=z_score, bins=60, linewidth=0.5, edgecolor="black")
plt.title("Datos transformados por z-score")
plt.show()

In [ ]:
z_score_abs = abs(z_score)
outliers_zscore = z_score_abs > 3 #+- 3 veces
outliers_zscore

In [ ]:
df_taxis[~outliers_zscore]['total'] 

In [ ]:
plt.figure(figsize=(5,4))
sns.histplot(data=df_taxis[~outliers_zscore]['total'], bins=60, linewidth=0.5, edgecolor="black")
plt.show()

In [ ]:
def Metodo_Z_score (df,n,features):
    outlier_list = []
    
    for column in features:
        # Calculando la media y la desviación estándar de cada característica
        data_mean = df[column].mean()
        data_std = df[column].std()
        threshold = 3
        
        z_score = abs( (df[column] - data_mean)/data_std )
        
        # Determinando los indices que corresponden a los outliers 
        outlier_list_column =  df[z_score > threshold].index
        
        # Añadiendo los indices que corresponden a los outliers
        outlier_list.extend(outlier_list_column)
        
    # Seleccionando los indices que cumplen con un numero n de veces que aparecen como outliers en cada columna
    outlier_list = Counter(outlier_list)        
    multiple_outliers = list( k for k, v in outlier_list.items() if v > n )
    
    return multiple_outliers

In [ ]:
df_taxis.head(1)

In [ ]:
df_taxis.columns

In [ ]:
df_taxis.select_dtypes(include='number').head(1) # Columnas numericas

In [ ]:
lista_caracteristicas = df_taxis.select_dtypes(include='number').columns.to_list()
lista_caracteristicas

In [ ]:
outliers_z_score = Metodo_Z_score(df_taxis,1,lista_caracteristicas)

# dropping outliers
df_woutliers_taxi = df_taxis.drop(outliers_z_score, axis = 0).reset_index(drop=True)

In [ ]:
df_woutliers_taxi.head()

In [ ]:
from scipy.stats import zscore

z_scores = zscore(df_taxis['total'])
abs_z_scores = np.abs(z_scores)
df_taxis[abs_z_scores >3]['total']

La elección de 3 como umbral proviene de la regla empírica, según la cual los datos dentro de 3 veces la desviación estándar respecto a la media representan el 99.7% de los datos de la distribución. Sabiendo esto, podemos concluir con bastante seguridad que los datos que caen más allá de este umbral son atípicos, pues son distintos al 99.7% de los datos.

###  Z-Score Modificado

Cuando los datos son asimétricos o no se distribuyen de forma normal podemos utilizar el **z-score modificado (MAD-Z Score)**. el z-score modificado mide cuánto se aleja un valor de la mediana en términos de la desviación absoluta mediana.

$$ M_i = \frac{0.6745*(x_i - Mediana)}{MAD}$$

donde:
- $x_i$: Un valor de dato individual
- $Mediana$: La mediana del conjunto de datos
- $MAD$: La desviación absoluta mediana del conjunto de datos

La desviación absoluta mediana ($MAD$) es una estadística robusta de variabilidad que mide la dispersión de un conjunto de datos. Es menos afectada por valores atípicos que otras medidas de dispersión como la desviación estándar y la varianza. 

Si los datos son normales, la desviación estándar suele ser la mejor opción para evaluar la dispersión. Sin embargo, si los datos no son normales, el MAD es una estadística que puedes usar en su lugar.

$$MAD = Mediana(|x_i – x_m|)$$

donde:
- $x_i$: El i-ésimo valor en el conjunto de datos
- $x_m$: El valor mediano en el conjunto de datos

**Ejemplo:** Considere los datos $(1, 1, 2, 2 , 4, 6, 9)$. Su mediana es $2$. Las desviaciones absolutas con respecto a $2$ son $(1, 1, 0, 0, 2, 4, 7)$, que a su vez tienen una mediana de $1$ (ya que las desviaciones absolutas ordenadas son $(0, 0, 1, 1 , 2, 4, 7)$). Por lo tanto, la desviación absoluta mediana de estos datos es $1$.

In [ ]:
df_taxis.head(1)

In [ ]:
median = df_taxis['total'].median() #caluclando la mediana
median

In [ ]:
abs_diff = (df_taxis['total'] - median).abs() #desviaciones absolutas con respecto a la mediana
abs_diff.median()

In [ ]:
from scipy.stats import median_abs_deviation #Usanndo la librería de python
mad_score = median_abs_deviation(df_taxis['total'])
mad_score

In [ ]:
def Metodo_Z_ScoreMod (df,n,features):
    outlier_list = []
    
    for column in features:
        # Calculando la media y la desviación estándar de cada característica
        data_mean = df[column].mean()
        data_std = df[column].std()
        threshold = 3
        MAD = median_abs_deviation #función de scipy.stats
        
        mod_z_score = abs(0.6745*(df[column] - data_mean)/MAD(df[column]) )
                
        # determinando la lista de los indices de outliers por cada característica   
        outlier_list_column =  df[mod_z_score >threshold].index
        
        # agregando los indices de los outliers de cada columna a la lista
        outlier_list.extend(outlier_list_column)
        
    # seleccionando las observaciones que contengan más de n número de outliers
    outlier_list = Counter(outlier_list)    
    #print(outlier_list)
    multiple_outliers = list( k for k, v in outlier_list.items() if v > n )
    
    return multiple_outliers

In [ ]:
# detectando outliers mediante el método z-score modificado
outliers_z_score = Metodo_Z_ScoreMod(df_taxis,2,lista_caracteristicas)

# eliminando los outliers del dataframe
df_out_zscoremod_taxis = df_taxis.drop(outliers_z_score, axis = 0).reset_index(drop=True)
df_out_zscoremod_taxis.head()

In [ ]:
len(df_out_zscoremod_taxis), len(df_taxis)

In [ ]:
#!pip install pyod
from pyod.models.mad import MAD
mad = MAD(threshold = 3)
lab = mad.fit(df_taxis['total'].values.reshape(-1,1)).labels_
lab

In [ ]:
sum(lab)

In [ ]:
df_taxis_woutliers = df_taxis[lab == 0]
df_taxis_woutliers.shape

In [ ]:
df_taxis_woutliers.head()

### Outliers Multivariados

Son valores atípicos detectados considerando la relación entre dos o más variables. Para este caso, se utilizan algoritmos de machine learning que ayudad a la detección de outliers, algunas metodologías más comunes para detectar valores atípicos multivariados son:

1. Local Outlier Factor (LOF)
2. Isolation Forest
3. DBSCAN



### Local Outlier Factor (LOF)

El método Local Outlier Factor (LOF) es un algoritmo no supervisado basado en la densidad local para detectar anomalías. LOF compara la densidad local de un punto con la de sus vecinos para determinar si es un outlier.

[LOF paper](https://dl.acm.org/doi/pdf/10.1145/335191.335388)

**¿Cómo funciona LOF?**

LOF mide la rareza de un punto calculando la densidad local de sus vecinos más cercanos. Si la densidad de un punto es significativamente menor que la de sus vecinos, se considera un outlier.

El algoritmo sigue estos pasos:

1. Para cada punto, encuentra sus k vecinos más cercanos.
2. Calcula la distancia de alcance de cada punto respecto a sus vecinos.
3. Calcula la densidad local de cada punto.
4. Calcula el factor LOF como la razón entre la densidad de los vecinos y la densidad del punto.

Un valor LOF cercano a 1 indica que el punto tiene una densidad similar a la de sus vecinos (no es un outlier). Un valor mucho mayor que 1 indica que el punto es un outlier.

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

# Generar datos normales
np.random.seed(42)
X_inliers = 0.3 * np.random.randn(100, 2)
X_inliers = np.r_[X_inliers + 2, X_inliers - 2]

In [ ]:
# Generar outliers
X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))

In [ ]:
# Unir datos
X = np.r_[X_inliers, X_outliers]

In [ ]:
# Ajustar LOF
clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
y_pred = clf.fit_predict(X)
outliers = y_pred == -1

# Visualización
plt.figure(figsize=(8,6))
plt.scatter(X[:, 0], X[:, 1], color='b', s=20, label='Normal')
plt.scatter(X[outliers, 0], X[outliers, 1], color='r', s=40, label='Outlier')
plt.title('Detección de anomalías con LOF')
plt.legend()
plt.show()

In [ ]:
#Ejemplo dataser heart
df_heart = pd.read_csv("Data/heart.csv")
df_heart.head()

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=50, contamination='auto')
X = df_heart[['Age','Chol']].values
y_pred = clf.fit_predict(X)

In [ ]:
y_pred

In [ ]:
plt.figure(figsize=(6,4))
in_mask = [True if l == 1 else False for l in y_pred]
out_mask = [True if l == -1 else False for l in y_pred]

plt.title("Local Outlier Factor (LOF)")

a = plt.scatter(X[in_mask, 0], X[in_mask, 1], c = 'blue',
                edgecolor = 'k', s = 30)
# outliers
b = plt.scatter(X[out_mask, 0], X[out_mask, 1], c = 'red',
                edgecolor = 'k', s = 30)
plt.axis('tight')
plt.xlabel('Age');
plt.ylabel('Cholestrol');
plt.show()

- LOF Detecta outliers locales y globales. No requiere suposiciones sobre la distribución de los datos.

**Sin embargo:**
- Es sensible a la elección de k (número de vecinos).
- Puede ser costoso computacionalmente para grandes volúmenes de datos.


### Isolation Forest

Isolation Forest (IF) es similar a Random Forest y se basa en árboles de decisión. No hay etiquetas predefinidas aquí. Es un algoritmo de aprendizaje no supervisado que identifica anomalías aislando valores atípicos en los datos.

Divide los datos usando un conjunto de árboles y proporciona una puntuación de anomalía observando cuán aislado está el punto en la estructura encontrada. La puntuación de anomalía se utiliza para diferenciar los valores atípicos de las observaciones normales. Un concepto importante en este método es el número de aislamiento. El número de aislamiento es el número de divisiones necesarias para aislar un punto de datos. Este número de divisiones se determina siguiendo estos pasos:

- Se selecciona aleatoriamente un punto “a” para aislar.
- Se selecciona aleatoriamente un punto de datos “b” que esté entre el valor mínimo y máximo y que sea diferente de “a”.
- Si el valor de “b” es menor que el de “a”, “b” se convierte en el nuevo límite inferior.
- Si el valor de “b” es mayor que el de “a”, “b” se convierte en el nuevo límite superior.
- Este procedimiento se repite mientras haya puntos de datos distintos de “a” entre el límite superior e inferior.

Se requieren menos divisiones para aislar un valor atípico que para aislar un valor no atípico, es decir, un valor atípico tiene un número de aislamiento menor en comparación con un punto no atípico. Por lo tanto, un punto de datos se define como valor atípico si su número de aislamiento es menor que el umbral. El umbral se define en función del porcentaje estimado de valores atípicos en los datos, que es el punto de partida de este algoritmo de detección de valores atípicos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest


In [ ]:
# Generar datos normales
rng = np.random.RandomState(42)
X = 0.3 * rng.randn(100, 2)
X = np.r_[X + 2, X - 2]

In [ ]:
# Generar outliers
X_outliers = rng.uniform(low=-4, high=4, size=(20, 2))

In [ ]:
# Unir datos
X_total = np.r_[X, X_outliers]

In [ ]:
# Ajustar Isolation Forest
clf = IsolationForest(contamination=0.1, random_state=42) #El parámetro `contamination` indica la proporción esperada de outliers
y_pred = clf.fit_predict(X_total)

# Visualización
plt.figure(figsize=(8,6))
plt.scatter(X_total[:, 0], X_total[:, 1], c=(y_pred==-1), cmap='coolwarm', s=40, edgecolors='k')
plt.title('Detección de outliers con Isolation Forest')
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

In [ ]:
#Ejemplo dataser california housing
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing(as_frame=True)
df = data.frame

# Seleccionar dos variables para visualizar
X_real = df[['AveRooms', 'AveOccup']].values

# Ajustar Isolation Forest
clf_real = IsolationForest(contamination=0.05, random_state=0)
y_pred_real = clf_real.fit_predict(X_real)

# Visualización
plt.figure(figsize=(8,6))
plt.scatter(X_real[:,0], X_real[:,1], c=(y_pred_real==-1), cmap='coolwarm', s=10, alpha=0.5)
plt.xlabel('AveRooms')
plt.ylabel('AveOccup')
plt.title('Outliers en California Housing (Isolation Forest)')
plt.show()

### DBSCAN

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) es un algoritmo de clustering basado en densidad, agrupa puntos que están densamente conectados y considera como outliers (ruido) aquellos puntos que no pertenecen a ningún grupo. Es como K-means, excepto que no es necesario especificar el número de grupos de antemano.

Parámetros principales:
- `eps`: Radio de vecindad para considerar puntos vecinos.
- `min_samples`: Número mínimo de puntos para formar un cluster (incluyendo el propio punto).

In [ ]:
from sklearn.cluster import DBSCAN

# Generar datos de dos clusters
rng = np.random.RandomState(0)
X1 = rng.normal(loc=[2, 2], scale=0.5, size=(50, 2))
X2 = rng.normal(loc=[-2, -2], scale=0.5, size=(50, 2))

In [ ]:
# Generar outliers
X_outliers = rng.uniform(low=-6, high=6, size=(10, 2))

# Unir datos
X = np.vstack([X1, X2, X_outliers])

In [ ]:
# Aplicar DBSCAN
db = DBSCAN(eps=0.7, min_samples=5)
labels = db.fit_predict(X)


In [ ]:
# Visualización
plt.figure(figsize=(8,6))
unique_labels = set(labels)
colors = [plt.cm.tab10(each) for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Outliers
        col = (1, 0, 0, 1)
        label = 'Outlier'
    else:
        label = f'Cluster {k}'
    class_member_mask = (labels == k)
    plt.plot(X[class_member_mask, 0], X[class_member_mask, 1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=10, label=label)
plt.title('DBSCAN: clusters y outliers')
plt.legend()
plt.show()

In [ ]:
# Ejemplo dataset insurance
df_3 = pd.read_csv("Data/insurance.csv")
df_3.head()

In [ ]:
from sklearn.cluster import DBSCAN
X = df_3[['age','bmi']].values

db = DBSCAN(eps=3.0, min_samples=10).fit(X)
labels = db.labels_

In [ ]:
pd.Series(labels).value_counts() #-1 representan valores atipicos

In [ ]:
unique_labels = set(labels)
colors = ['blue', 'red']

plt.figure(figsize=(6,4))
for color,label in zip(colors, unique_labels):
    sample_mask = [True if l == label else False for l in labels]
    plt.plot(X[:,0][sample_mask], X[:, 1][sample_mask], '*', color=color);
plt.xlabel('Age');
plt.ylabel('BMI');
plt.show()

## ¿Qué hacer después de identificar los outliers?

> **Mantenerlos:** Podemos mantener los outliers si consideramos que pueden ser representativos de un subconjunto de nuestros datos.

> **Eliminarlos:** Si estamos seguros de que los outliers provienen de un error en la entrada de los datos, como un error humano o de medida, y no podemos solucionarlo, podemos eliminarlos del conjunto de datos.

> **Imputar:** La imputación implica reemplazar los valores atípicos con otros valores como la mediana o la media. Esto se suele hacer cuando queremos conservar la mayor cantidad de datos, pero eliminando el efecto de los outliers.


In [ ]:
yout = y.reshape(-1,1)

In [ ]:
yout

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(x,y,'xr', label='datos')
plt.grid()
plt.show()

In [ ]:
lin2 = LinearRegression()

In [ ]:
lin2.fit(xi, yout)

In [ ]:
lin2.intercept_[0], lin2.coef_[0][0]

In [ ]:
betaout = [lin2.intercept_[0], lin2.coef_[0][0]]
betaout

In [ ]:
# yout = b0 + b1*x
y_fit = beta[0] + beta[1]*x
y_fitout = betaout[0] + betaout[1]*x
plt.figure(figsize=(6,4))
plt.plot(x,y,'xr', label='datos')
plt.plot(x,y_fit, 'b', lw=3)
plt.plot(x,y_fitout, 'g', lw=3)
plt.grid()
plt.show()